In [2]:
!pip install shutup #optional :>
import shutup
shutup.please()

# **Import necessary libraries**

In [3]:
!pip install yfinance  #financial market data
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import re
import datetime #datetime data type
import yfinance as yf
%matplotlib inline
sb.set()

# **Data Preparation and Cleaning**

# ***Trumps tweets***
- Trumps tweets dated from 1st Jan 2015 to 16 June 2020 was retrieved from https://www.kaggle.com/austinreese/trump-tweets 


In [4]:
trumpTweet = pd.read_csv('https://raw.githubusercontent.com/calebczh/Trump-Tweets-and-Financial-Market-Relationship/main/Datasets/realdonaldtrump.csv')
trumpTweet

,id,link,content,date,retweets,favorites,mentions,hashtags
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,NaN,NaN
...,...,...,...,...,...,...,...,...
43347,1273405198698975232,https://twitter.com/realDonaldTrump/status/127...,Joe Biden was a TOTAL FAILURE in Government. H...,2020-06-17 19:00:32,23402,116377,NaN,NaN
43348,1273408026968457216,https://twitter.com/realDonaldTrump/status/127...,Will be interviewed on @ seanhannity tonight a...,2020-06-17 19:11:47,11810,56659,@seanhannity,NaN
43349,1273442195161387008,https://twitter.com/realDonaldTrump/status/127...,pic.twitter.com/3lm1spbU8X,2020-06-17 21:27:33,4959,19344,NaN,NaN
43350,1273442469066276864,https://twitter.com/realDonaldTrump/status/127...,pic.twitter.com/vpCE5MadUz,2020-06-17 21:28:38,4627,17022,NaN,NaN


Description of the dataset is as follows.


> **Id** : Unique tweet ID.

> **link** : Link to the tweet.

> **content** : Content of the tweet.

> **date** : The date and the time that the tweet was posted on Twitter.

> **retweets** : This column contains the amount of retweets for the tweet. A retweet is a re-posting of a Tweet by other Twitter users. 

> **favorites** : This column contains the amount of favorites for the tweet. Favorites are described as indicators that a tweet is well-liked or popular among online users. A tweet can be identified as a favorite by the small star icon seen beside the post. Twitter users can mark a tweet as a favorite to let the author know that someone likes it.

> **mentions** : A mention in a Tweet is another person's username anywhere in the body of the Tweet. The user mentioned will be notified and tagged on the tweet.

> **hashtags** : When you use a hashtag in a Tweet, it becomes linked to all of the other Tweets that include it. Including a hashtag gives your Tweet context and allows people to easily follow topics that they're interested in.

In [5]:
#removing column id and link as they do not provide any possibily insightful infromation
trumpTweet = trumpTweet.drop("link", axis = 1) 
trumpTweet = trumpTweet.drop("id", axis = 1)

#splitting the date into date and time 
trumpTweet[['date', 'time']] = trumpTweet['date'].str.split(' ', 1, expand=True) 
trumpTweet['content'] = trumpTweet['content'].replace(r'http\S+', ' ', regex=True).replace(r'www\S+', ' ', regex=True).replace(r'pic.twitter\S+', ' ', regex=True).replace(r'@ \S+', ' ', regex=True) #removing urls, mentions and hashtags in contents 

#checking if a row's content only contained url
trumpTweet['content_isspace'] = trumpTweet['content'].apply(lambda x: x.isspace()) 

#deleting those within only urls as they do not provide any insight
trumpTweet = trumpTweet[trumpTweet.content_isspace == False] 
trumpTweet = trumpTweet.drop('content_isspace', axis = 1) 

#setting date to datetime data type
trumpTweet['date'] = pd.to_datetime(trumpTweet['date']) 

#setting date as the index
trumpTweet = trumpTweet.set_index('date') 
trumpTweet

,content,retweets,favorites,mentions,hashtags,time
date,,,,,,
2009-05-04,Be sure to tune in and watch Donald Trump on L...,510,917,NaN,NaN,13:54:25
2009-05-04,Donald Trump will be appearing on The View tom...,34,267,NaN,NaN,20:00:10
2009-05-08,Donald Trump reads Top Ten Financial Tips on L...,13,19,NaN,NaN,08:38:08
2009-05-08,New Blog Post: Celebrity Apprentice Finale and...,11,26,NaN,NaN,15:40:15
2009-05-12,"""My persona will never be that of a wallflower...",1375,1945,NaN,NaN,09:07:28
...,...,...,...,...,...,...
2020-06-16,True! …,26783,154978,NaN,NaN,20:16:20
2020-06-16,A GREAT woman. Her son is looking down from he...,26468,112140,NaN,NaN,21:31:11
2020-06-16,96% Approval Rating in the Republican Party. T...,44472,313534,NaN,NaN,22:27:56


# ***S&P 500***
- S&P 500 Historical Data retrieved from yFinance library.

In [6]:
start = "2009-05-03"  #one day before the first tweet in the dataset so as to calculate to %cahnge in teh financial market with respect to the preevious day
end = '2020-06-17' #last tweet's date of the dataset
sp500 = yf.download('^GSPC',start,end)
sp500

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-05-04,879.210022,907.849976,879.210022,907.239990,907.239990,7038840000
2009-05-05,906.099976,907.700012,897.340027,903.799988,903.799988,6882860000
2009-05-06,903.950012,920.280029,903.950012,919.530029,919.530029,8555040000
2009-05-07,919.580017,929.580017,901.359985,907.390015,907.390015,9120100000
2009-05-08,909.030029,930.169983,909.030029,929.229980,929.229980,8163280000
...,...,...,...,...,...,...
2020-06-10,3213.419922,3223.270020,3181.489990,3190.139893,3190.139893,6570840000
2020-06-11,3123.530029,3123.530029,2999.489990,3002.100098,3002.100098,7018890000
2020-06-12,3071.040039,3088.419922,2984.469971,3041.310059,3041.310059,5832250000


Description of the dataset is as follows.


> **Open** : Open is the price at which a stock began trading in the period.

> **High** : High refers to the maximum price in the given trading period.

> **Low** : Low refers to the minimum price in the given trading period.

> **Close** : Close is the price at which a stock stopped trading in the period.

> **Adj Close** : This contains adjusted close which is the closing price after adjustments for all applicable splits and dividend distributions.

> **Volume** :  Volume refers to the number of shares traded in the given trading period.

A percentage increase in stock value is the change in stock comparing that to the previous day. The bigger the value either positive or negative the volatile the stock is. To measure the extent in which the stock is affect by Trump's tweets in the prvious day, we have to add in a column of percentage change in SP500. For this percentage, we will like to use the opening of the day over the closing of the previous day as effect of Trump's tweets can be reduced over time.

In [7]:
sp500['returns'] = (sp500['Close']/sp500['Close'].shift(1)) -1
sp500

,Open,High,Low,Close,Adj Close,Volume,returns
Date,,,,,,,
2009-05-04,879.210022,907.849976,879.210022,907.239990,907.239990,7038840000,NaN
2009-05-05,906.099976,907.700012,897.340027,903.799988,903.799988,6882860000,-0.003792
2009-05-06,903.950012,920.280029,903.950012,919.530029,919.530029,8555040000,0.017404
2009-05-07,919.580017,929.580017,901.359985,907.390015,907.390015,9120100000,-0.013202
2009-05-08,909.030029,930.169983,909.030029,929.229980,929.229980,8163280000,0.024069
...,...,...,...,...,...,...,...
2020-06-10,3213.419922,3223.270020,3181.489990,3190.139893,3190.139893,6570840000,-0.005313
2020-06-11,3123.530029,3123.530029,2999.489990,3002.100098,3002.100098,7018890000,-0.058944
2020-06-12,3071.040039,3088.419922,2984.469971,3041.310059,3041.310059,5832250000,0.013061
